In [34]:
#pip install dash
#pip install dash-leaflet

In [ ]:
#####################__Import of libraries__####################################
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_leaflet as dl
import requests
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import mapping
import leafmap as leafmap
from shapely.geometry import shape
from dash_extensions.javascript import assign

import branca.colormap as cm
import json
import matplotlib.colors as mcolors
import plotly.graph_objects as go
############################################# FUNCTION TO POPULATE THE DROPDOWN #############################################

# pollutant_dropdown populates the dropdown with the pollutatnts that have at least one recorded value

def pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list

# all_pollutant_dropdown populates the dropdown with all the pollutants, even those that have no recorded values

def all_pollutant_dropdown():
    t=requests.get(url="http://127.0.0.1:5000/api/all_pollutant")
    list_pollutant = read_response(t)
    list = [{'label': pollutant, 'value': pollutant} for pollutant in list_pollutant]
    return list



##############################_____API Call Support Functions_____####################################

def read_response(response):
    if response.status_code == 200:
        return response.json()
    return None

def get_pollutants():
    r = requests.get("http://127.0.0.1:5000/api/pollutants")
    return [{'label': p, 'value': p} for p in read_response(r) or []]  #Turn each pollutant into a dictionary

def get_municipalities():
    r = requests.get("http://127.0.0.1:5000/api/municipalities")
    return [{'label': m, 'value': m} for m in read_response(r) or []] #Turn each municipality into a dictionary

def get_provinces():
    r = requests.get("http://127.0.0.1:5000/api/provinces")
    return [{'label': p, 'value': p} for p in read_response(r) or []] #Turn each province into a dictionary

def get_dv8_data(pollutant, start_date, end_date):
    url = "http://127.0.0.1:5000/api/DV_8"
    payload = {"pollutant": pollutant, "start_date": start_date, "end_date": end_date}
    r = requests.post(url, json=payload)
    return pd.DataFrame(read_response(r) or []) #Makes a POST to the /api/DV_8 endpoint with a JSON that specifies the pollutant and date range, returning a DataFrame

def get_all_province_shapes():
    url = "http://127.0.0.1:5000/api/province_shape_v2"
    r = requests.get(url)
    data = read_response(r)
    if data:
        df = pd.DataFrame(data)
        df["geometry"] = df["geometry_province_wkt"].apply(wkt.loads)
        return gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
    else:
        return gpd.GeoDataFrame(columns=["nome_provincia", "geometry"], geometry="geometry", crs="EPSG:4326") #Makes a GET request to the /api/province_shape_v2 endpoint, returning a GeoDataFrame with the geometries of all provinces

# get_measurement_unit returns the measurement unit of a given pollutant

def get_measurement_unit(pollutant):
    t=requests.post(url="http://127.0.0.1:5000/api/units", json={"var_pollutant": pollutant}) #json= data will convert the dictionary to a json object and send it to the server
    data = read_response(t) 
    return data[0]

# Style function for dv9 
style_dv9 = assign("""
function(feature) {
    return {
        color: feature.properties.color_code || "blue",
        weight: 1,
        fillOpacity: 0.5
    };
}
""")

#tooltip on each feature
on_each_feature_dv9 = assign("""
function(feature, layer) {
    if (feature.properties && feature.properties.prov_upper) {
        layer.bindTooltip(feature.properties.prov_upper);
    }
}
""")

################################_____ Dash App Initialization ____####################################

app = Dash(__name__)

################################_____ Layout ____####################################

app.layout = html.Div([

    ############################################# DV_8 DV_9 #############################################
    html.Div([
        html.H2('DV_8 & DV_9: Average concentration by province'),
        html.P("This section enables users to compare the average pollutant concentrations across all provinces within a chosen time interval. Select a pollutant and define a start and end date to retrieve the relevant data. The output includes both a bar chart displaying average concentrations per province and map where provinces are colored based on whether their values are above or below the regional average. This dual visualization helps identify geographical disparities in air pollution levels and supports decision-making related to environmental monitoring or policy interventions."),

        # Dropdown pollutant
        dcc.Dropdown(
            id='pollutant_dropdown_dv8', #it will be used in the callback to fetch data
            options=pollutant_dropdown(),     # sostituito get_pollutants(),
            placeholder="Select pollutant",
            style={'margin-top': 20}
        ),

        dcc.DatePickerRange(
            id="date-range-picker_DV_8",
            start_date_placeholder_text="Start Date",
            end_date_placeholder_text="End Date",
            display_format="YYYY-MM-DD",
            style={"margin-top": 10}
            ),

        # Button to activate results
        html.Div([
            html.Button('Visualize/update histogram and map', id='submit_button_dv8', n_clicks=0, className='button')
        ], style={'textAlign': 'center'}),

        # Loading wrapper for bar chart and map 
        dcc.Loading(
            type="dot",
            children=[dcc.Graph(id='bar_chart_dv8')]
        ),

        html.Hr(style={"border": "1px solid blue", "opacity": 0.8, "margin": "30px 0"}),  # Divider

        dcc.Loading(
                type="dot",  
                #children=[dcc.Graph(id='map_dv8')]
                children=dl.Map(   #ALTERNATIVA CON dl.Map, ma così va modificata callback
                    id="map_dv9",
                   center=[45.64, 9.60],
                    zoom=7,
                    children=[dl.TileLayer(), dl.LayersControl(id="layers_map_dv9")],
                    style={"height": "50vh"},
                )
            )

    ], className='div_border'), 
])


########################################### Callbacks for DV_8 DV_9 #############################################

@app.callback(
        [Output('bar_chart_dv8', 'figure'), Output('layers_map_dv9', 'children')],
        Input('submit_button_dv8', 'n_clicks'),
        State('pollutant_dropdown_dv8', 'value'),
        State('date-range-picker_DV_8', 'start_date'),
        State('date-range-picker_DV_8', 'end_date')) #parte quando il bottone DV_8 viene premuto, legge gli input e aggiorna i grafici

def update_dv8(n_clicks, pollutant, start_date, end_date):
    
    if n_clicks == 0:
      return px.line(),[] # Return an empty figure if no button has been clicked
     
    if n_clicks > 0 and (not pollutant or not start_date or not end_date):
        return ( px.line().update_layout(
            annotations=[dict(
                    text="Please select a pollutant and a sensor to visualize the plot",
                    xref="paper", yref="paper",
                    showarrow=False,
                    font=dict(size=14, color="white"),
                    bgcolor="red",
                    borderpad=5)]), [])
    
        
    df = get_dv8_data(pollutant, start_date, end_date) #richiama la funzione per ottenere i dati DV_8 e restituisce un DataFrame con due colonne: 'province' e 'average_value
    unit = get_measurement_unit(pollutant)

    if df.empty:
        return px.line(title=f"No data found for pollutant {pollutant} between '{start_date}' and '{end_date}'."), []

    provinces_gdf = get_all_province_shapes() #carica un geodataframe con le geometrie delle province
    

    df['prov_upper'] = df['province'].str.upper().str.strip() # Normalizza i nomi delle province in maiuscolo e senza spazi

    provinces_gdf['prov_upper'] = provinces_gdf['nome_provincia'].str.upper().str.strip() # Normalizza i nomi delle province nel geodataframe
    merged = provinces_gdf.merge(df, on='prov_upper', how='left') # Unisce i dati delle province con i dati DV_8

    media_globale = merged.loc[merged['average_value'].notnull(), 'average_value'].mean() # Calcola la media globale solo sulle province con valori validi

    def color_group(row): # Funzione per assegnare il codice colore in base al valore medio
        if pd.isnull(row['average_value']):
            return 'grey'
        elif row['average_value'] < media_globale:
            return 'blue'
        else:
            return 'red' 
        

    merged['color_code'] = merged.apply(color_group, axis=1) # Applica la funzione per creare una nuova colonna 'color_code' che indica il colore da usare


    bar_fig = px.bar(merged, x='province', y='average_value',
                     title=f"Average concentration of {pollutant} by province",
                     labels={'average_value': f'Average concentration of {pollutant} [{unit}]', 'province': 'Province'},
                     color = 'color_code',
                     color_discrete_map={
                                        'red': 'red',
                                        'blue': 'blue',
                                        'grey': 'grey'}) # Crea il grafico a barre con i valori medi per provincia
                    
    bar_fig.update_layout(showlegend=False) #disattiva la legenda

    bar_fig.add_hline(y=media_globale, line_dash='dash', line_color='blue',
                      annotation_text=f"Regional average: {media_globale:.2f}",
                      annotation_position="top left") # Aggiunge una linea orizzontale per la media regionale
    
    merged['geometry'] = merged['geometry'].simplify(tolerance=0.001, preserve_topology=True) #per velocizzare conversione
    #geojson_dv9 = json.loads(merged.to_json()) # Converte il GeoDataFrame in un formato GeoJSON per la mappa (quindi leggibile da Plotly)
    geojson_dv9 = json.loads(merged[['geometry', 'prov_upper', 'color_code']].to_json()) #Converte solo feature utili il GeoDataFrame in un formato GeoJSON per la mappa
    

    

########

    map_layers = [
        dl.Overlay(
                dl.GeoJSON(
                    data=geojson_dv9,  # Path to the GeoJSON file
                    options={"style": style_dv9, "onEachFeature": on_each_feature_dv9},
                    #fillColor = merged['color_code'],
                    hoverStyle=dict(weight=3),
                    zoomToBoundsOnClick=True,
                    
                    
                 ),
                name="Provinces",
                checked=True
            ),

            

            html.Div([
                html.Div([
                    html.Span(style={
                        "backgroundColor": "lightgrey",
                        "display": "inline-block",
                        "width": "15px",
                        "height": "15px",
                        "marginRight": "5px",
                        "border": "1px solid black"
                    }),
                    html.Span("No data")
                ], style={"marginBottom": "5px"}),

                html.Div([
                    html.Span(style={
                        "backgroundColor": "blue",
                        "display": "inline-block",
                        "width": "15px",
                        "height": "15px",
                        "marginRight": "5px",
                        "border": "1px solid black"
                    }),
                    html.Span("Below average")
                ], style={"marginBottom": "5px"}),

                html.Div([
                    html.Span(style={
                        "backgroundColor": "red",
                        "display": "inline-block",
                        "width": "15px",
                        "height": "15px",
                        "marginRight": "5px",
                        "border": "1px solid black"
                    }),
                    html.Span("Above average")
                ])
            ], style={
                "position": "absolute",
                "bottom": "10px",
                "left": "10px",
                "padding": "5px",
                "zIndex": "999"}
            )

    ]
    return bar_fig, map_layers

   

if __name__ == '__main__':
    app.run(debug=True)
